# Validation GPU Phase 4.1 - Correction CFL

**Objectif** : Valider la correction CFL et tester la précision GPU vs CPU

- ✅ Simulation CPU de référence
- ✅ Simulation GPU avec correction CFL
- ✅ Export des résultats dans `output_gpu.zip`
- 📊 **Analyse en local** avec scripts séparés

## 1. Setup Environnement Kaggle

In [ ]:
# Cloner le dépôt et setup
!git clone https://github.com/elonmj/Projet_tutore_ARZ.git

import os
import shutil
import numpy as np
from datetime import datetime

# Copier les fichiers nécessaires
ITEMS_TO_COPY = ['code', 'config']
for item in ITEMS_TO_COPY:
    source_path = f'Projet_tutore_ARZ/{item}'
    if os.path.exists(source_path):
        if os.path.exists(item):
            shutil.rmtree(item)
        shutil.copytree(source_path, item)
        print(f"✅ {item} copié")

print(f"📁 Répertoire: {os.getcwd()}")

## 2. Vérification CUDA

In [ ]:
# Vérification CUDA
try:
    from numba import cuda
    if cuda.is_available():
        device = cuda.get_current_device()
        print(f"✅ GPU: {device.name}")
        print(f"   Capacité: {device.compute_capability}")
    else:
        print("❌ CUDA non disponible")
except ImportError:
    print("❌ Numba CUDA non trouvé")

## 3. Simulation CPU (Référence)

In [ ]:
# Simulation CPU de référence
print("🖥️ SIMULATION CPU (référence avec correction CFL)")
print("="*50)

start_time = datetime.now()

try:
    from code.simulation.runner import SimulationRunner
    
    runner_cpu = SimulationRunner(
        'config/scenario_gpu_validation.yml',
        device='cpu',
        quiet=False
    )
    
    times_cpu, states_cpu = runner_cpu.run()
    
    end_time = datetime.now()
    duration_cpu = (end_time - start_time).total_seconds()
    
    print(f"✅ Simulation CPU terminée en {duration_cpu:.1f}s")
    print(f"   Forme résultats: times={times_cpu.shape}, states={states_cpu.shape}")
    
    # Vérifier les infos CFL
    if hasattr(runner_cpu.params, '_cfl_debug'):
        cfl_info = runner_cpu.params._cfl_debug
        print(f"   CFL final: {cfl_info.get('last_cfl', 'N/A'):.3f}")
        print(f"   dt: {cfl_info.get('last_dt_corrected', 'N/A'):.6e}s")
    
    cpu_success = True
    
except Exception as e:
    print(f"❌ Erreur CPU: {e}")
    cpu_success = False
    times_cpu, states_cpu = None, None

## 4. Simulation GPU (Test)

In [ ]:
# Simulation GPU avec correction CFL
print("🚀 SIMULATION GPU (avec correction CFL)")
print("="*50)

if cpu_success:
    start_time = datetime.now()
    
    try:
        runner_gpu = SimulationRunner(
            'config/scenario_gpu_validation.yml',
            device='gpu',
            quiet=True
        )
        
        times_gpu, states_gpu = runner_gpu.run()
        
        end_time = datetime.now()
        duration_gpu = (end_time - start_time).total_seconds()
        
        print(f"✅ Simulation GPU terminée en {duration_gpu:.1f}s")
        print(f"   Forme résultats: times={times_gpu.shape}, states={states_gpu.shape}")
        
        # Speedup
        if duration_cpu > 0:
            speedup = duration_cpu / duration_gpu
            print(f"   🚀 Speedup: {speedup:.2f}x")
        
        # Vérifier les infos CFL
        if hasattr(runner_gpu.params, '_cfl_debug'):
            cfl_info = runner_gpu.params._cfl_debug
            print(f"   CFL final: {cfl_info.get('last_cfl', 'N/A'):.3f}")
            print(f"   dt: {cfl_info.get('last_dt_corrected', 'N/A'):.6e}s")
        
        gpu_success = True
        
    except Exception as e:
        print(f"❌ Erreur GPU: {e}")
        gpu_success = False
        times_gpu, states_gpu = None, None
else:
    print("⚠️ Simulation GPU ignorée (échec CPU)")
    gpu_success = False
    times_gpu, states_gpu = None, None

## 5. Comparaison Rapide

In [ ]:
# Comparaison rapide CPU vs GPU
if cpu_success and gpu_success:
    print("🔍 COMPARAISON RAPIDE CPU vs GPU")
    print("="*40)
    
    # Vérifier compatibilité des formes
    if times_cpu.shape == times_gpu.shape and states_cpu.shape == states_gpu.shape:
        # Calcul erreurs
        diff_states = np.abs(states_cpu - states_gpu)
        
        error_max = np.max(diff_states)
        error_mean = np.mean(diff_states)
        
        print(f"📊 Erreur maximale: {error_max:.3e}")
        print(f"📊 Erreur moyenne: {error_mean:.3e}")
        
        # Évaluation
        if error_max < 1e-10:
            print("🟢 EXCELLENT: Précision < 1e-10")
            status = "EXCELLENT"
        elif error_max < 1e-8:
            print("🟢 TRÈS BON: Précision < 1e-8")
            status = "TRÈS BON"
        elif error_max < 1e-6:
            print("🟡 ACCEPTABLE: Précision < 1e-6")
            status = "ACCEPTABLE"
        else:
            print("🔴 PROBLÉMATIQUE: Précision > 1e-6")
            status = "PROBLÉMATIQUE"
        
        # Comparaison avant/après correction CFL
        print(f"\n📈 vs AVANT CORRECTION CFL:")
        print(f"   Avant: ~1e-3 (CFL=34.924)")
        print(f"   Après: {error_max:.3e} (CFL≤0.5)")
        
        if error_max < 1e-3:
            improvement = 1e-3 / error_max
            print(f"   🎉 Amélioration: {improvement:.0f}x")
        
        comparison_success = True
        
    else:
        print("❌ Formes incompatibles CPU/GPU")
        print(f"   CPU: times={times_cpu.shape}, states={states_cpu.shape}")
        print(f"   GPU: times={times_gpu.shape}, states={states_gpu.shape}")
        comparison_success = False
        error_max, error_mean, status = None, None, "ÉCHEC"
        
else:
    print("⚠️ Comparaison impossible (échec simulation)")
    comparison_success = False
    error_max, error_mean, status = None, None, "ÉCHEC"

## 6. Export des Résultats

In [ ]:
# Export des résultats pour analyse locale
import zipfile
import json

print("📦 EXPORT DES RÉSULTATS")
print("="*30)

# Créer dossier output
output_dir = "output_gpu"
os.makedirs(output_dir, exist_ok=True)

# Timestamp pour les fichiers
timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")

files_exported = []

# Sauvegarder résultats CPU
if cpu_success:
    cpu_file = f"{output_dir}/results_cpu_{timestamp}.npz"
    np.savez_compressed(cpu_file, 
                       times=times_cpu, 
                       states=states_cpu,
                       grid_info=runner_cpu.grid,
                       params_info=runner_cpu.params)
    files_exported.append(cpu_file)
    print(f"✅ CPU: {cpu_file}")

# Sauvegarder résultats GPU
if gpu_success:
    gpu_file = f"{output_dir}/results_gpu_{timestamp}.npz"
    np.savez_compressed(gpu_file, 
                       times=times_gpu, 
                       states=states_gpu,
                       grid_info=runner_gpu.grid,
                       params_info=runner_gpu.params)
    files_exported.append(gpu_file)
    print(f"✅ GPU: {gpu_file}")

# Métadonnées de la validation
metadata = {
    "timestamp": timestamp,
    "cpu_success": cpu_success,
    "gpu_success": gpu_success,
    "comparison_success": comparison_success,
    "cpu_duration": duration_cpu if cpu_success else None,
    "gpu_duration": duration_gpu if gpu_success else None,
    "speedup": duration_cpu/duration_gpu if cpu_success and gpu_success else None,
    "error_max": float(error_max) if error_max is not None else None,
    "error_mean": float(error_mean) if error_mean is not None else None,
    "status": status,
    "phase": "4.1",
    "correction_cfl": "active",
    "objective": "< 1e-10"
}

metadata_file = f"{output_dir}/validation_metadata_{timestamp}.json"
with open(metadata_file, 'w') as f:
    json.dump(metadata, f, indent=2)
files_exported.append(metadata_file)
print(f"✅ Métadonnées: {metadata_file}")

# Informations de configuration
config_info = {
    "scenario": "scenario_gpu_validation.yml",
    "grid": {"N": 200, "xmin": 0.0, "xmax": 1000.0, "dx": 5.0},
    "temporal": {"t_final": 10.0, "output_dt": 1.0},
    "numerics": {"cfl_number": 0.4, "spatial_scheme": "weno5", "time_scheme": "ssprk3"},
    "correction_cfl": "enabled"
}

config_file = f"{output_dir}/config_info_{timestamp}.json"
with open(config_file, 'w') as f:
    json.dump(config_info, f, indent=2)
files_exported.append(config_file)
print(f"✅ Config: {config_file}")

print(f"\n📁 Fichiers dans {output_dir}:")
for file in files_exported:
    size = os.path.getsize(file) / 1024  # KB
    print(f"   {file} ({size:.1f} KB)")

## 7. Création ZIP Final

In [ ]:
# Créer le ZIP final (nom fixe pour écraser l'ancien)
zip_filename = "output_gpu.zip"

with zipfile.ZipFile(zip_filename, 'w', zipfile.ZIP_DEFLATED) as zipf:
    for file in files_exported:
        zipf.write(file, os.path.basename(file))

zip_size = os.path.getsize(zip_filename) / (1024*1024)  # MB

print(f"📦 ZIP créé: {zip_filename} ({zip_size:.2f} MB)")
print(f"\n📥 INSTRUCTIONS:")
print(f"1. Télécharger {zip_filename} depuis l'onglet Output")
print(f"2. Extraire en local")
print(f"3. Utiliser les scripts d'analyse locaux")

# Résumé final
print(f"\n🎯 RÉSUMÉ VALIDATION PHASE 4.1")
print(f"="*40)
print(f"Simulation CPU: {'✅' if cpu_success else '❌'}")
print(f"Simulation GPU: {'✅' if gpu_success else '❌'}")
print(f"Comparaison: {'✅' if comparison_success else '❌'}")
if comparison_success:
    print(f"Statut: {status}")
    print(f"Erreur max: {error_max:.3e}")
print(f"Export: ✅ {zip_filename}")

print(f"\n✅ VALIDATION TERMINÉE - Prêt pour analyse locale")